In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from pathlib import Path
import os
import sys
import json
import copy
import pickle
from hashlib import sha256
import random

In [28]:
import pandas as pd
import pandas as pd
import numpy as np
import cv2
from tqdm.notebook import tqdm

In [29]:
sys.path.append('../lared')
from helpers import file_functions
from dataset.example import VideoExample, AudioExample, FullExample
from constants.laughter_constants import data_path, cloud_data_path
from data_loading.utils import get_video_caps, AudioFiles

In [30]:
VideoExample.init_caps(get_video_caps('/mnt/hdd/data/lared/concat'))

TypeError: get_video_caps() takes 0 positional arguments but 1 was given

In [31]:
laughter_ann_path = os.path.join(cloud_data_path, 'laughter.txt')
camera_ann_path = os.path.join(cloud_data_path, 'camera.txt')
out_path = os.path.join(cloud_data_path, 'examples')
images_path = os.path.join(out_path, 'images')


laughter_ann = pd.read_csv(laughter_ann_path, sep='\t',names=['name','participant','ini','fin','dur','type'],header=None)
camera_ann = pd.read_csv(camera_ann_path, sep='\t',names=['name','participant','ini','fin','dur','camera'],header=None)

In [32]:
camera_ann.head()

,name,participant,ini,fin,dur,camera
0,camera30,32.0,0.000,10.000,10.000,0
1,camera3,3.0,866.280,882.440,16.160,1
2,camera3,3.0,883.150,884.490,1.340,12
3,camera3,3.0,884.600,888.860,4.260,123
4,camera3,3.0,4300.139,4300.956,0.817,4


In [33]:
df_camera_by_participant = camera_ann.groupby('participant')
df_laughter_by_participant = laughter_ann.groupby('participant')

In [34]:
dfs_camera = []
for i in df_camera_by_participant.groups:
    dfs_camera.append(df_camera_by_participant.get_group(i))
dfs_laughter = []
for i in df_laughter_by_participant.groups:
    dfs_laughter.append(df_laughter_by_participant.get_group(i))
participant_ids = df_camera_by_participant.groups.keys()
participant_ids = [int(id) for id in participant_ids]

In [35]:
def get_camera(camerastr):
    if int(camerastr) == 0:
        return []
    
    return [int(c) for c in str(camerastr)]

In [36]:
# find the camera where a person is seen
def find_cameras(cam, t):
    previous = cam[cam['fin'] < t]
    if len(previous) == 0:
        return []
    
    row = previous.iloc[-1]
    cameras = row.tail()['camera']
    return get_camera(cameras)

# Laughter examples

In [37]:
# Create examples and first images
examples = dict()
for pid, df_laughter, df_camera in zip(participant_ids, dfs_laughter, dfs_camera):
    for index,row in df_laughter.iterrows():
        cameras = find_cameras(df_camera, row['ini'])
        ini_time = (row['ini'])
        fin_time = (row['fin'])

        for cam in cameras:

            hashstr = '_'.join([str(pid), str(cam), str(ini_time), str(fin_time)])
            hashid = sha256(hashstr.encode()).hexdigest()
            
            examples[hashid] = FullExample({
                'id': hashid,
                'pid': pid,
                'cam': cam,
                'ini_time': ini_time,
                'fin_time': fin_time
            })

            examples[hashid].write_full_first_image(images_path)

AttributeError: 'FullExample' object has no attribute 'write_full_first_image'

In [38]:
def create_random_segments(df_camera, pid, segment_len=10):
    examples = list()
    it = df_camera.iterrows()
    prev = next(it)[1]
    for index, row in it:
        ini = prev.loc['fin'] + 10
        fin = row.loc['ini'] - 10
        cameras = get_camera(prev.loc['camera'])

        for cam in cameras:
            t = ini
            while t < fin:
                if t < 14*60:
                    t += segment_len
                    continue

                hashstr = '_'.join([str(pid), str(cam), str(t), str(t+segment_len)])
                hashid = sha256(hashstr.encode()).hexdigest()
                examples.append(FullExample({
                    'id': hashid,
                    'pid': pid,
                    'cam': cam,
                    'ini_time': t,
                    'fin_time': t + segment_len
                }))
                t += segment_len
        prev = row
    return examples

In [40]:
random_examples = [create_random_segments(df, pid) for pid, df in zip(participant_ids, dfs_camera)]

In [41]:
random_examples = [ex for e in random_examples for ex in e]

In [42]:
# sample examples and put them in order of pid (for easy labeling)
random.seed(22)
chosen_examples = random.sample(random_examples, 2000)
chosen_examples = sorted(chosen_examples, key=lambda ex: ex.pid)

In [43]:
len(chosen_examples)

2000

# Write images

In [24]:
for ex in tqdm(chosen_examples[:2000]):
    ex.write_full_first_image(images_path)

KeyboardInterrupt: 

# Read CVAT annotations

In [44]:
examples_dict = {ex.id: ex for ex in random_examples}

In [45]:
cvat_bb_path = os.path.join(cloud_data_path, 'examples/pilot/cvat.json')

In [46]:
# Read the BBs from cvat annotations
# Read camera and bounding box from it
with open(cvat_bb_path) as json_file:
    bbs = json.load(json_file)

In [48]:
# create dict of img_id -> img
imgs_dict = dict()
for img in bbs['images']:
    imgs_dict[img['id']] = img

In [51]:
# iterate over annotations
for annot in bbs['annotations']:
    annot_img = imgs_dict[annot['image_id']]
    example_id = annot_img['file_name'].split('_')[1]
    example = examples_dict[example_id]

    bbox = annot['bbox']
    example.rect = [int(e) for e in bbox]

    example.write_video()
    example.write_audio()
    example.write_audiovisual()

ValueError: Unknown format code 'd' for object of type 'str'